In [1]:
from src.predictor import Predictor
import pandas as pd
from src.utils import add_top_k_scores_to_df, save_dataframe_to_xlsx

In [2]:
valid_df = pd.read_csv('./data/preprocessed/validation_dataset.csv')
master_file = pd.read_csv('./data/preprocessed/master_file.csv')

queries, query_prices = valid_df['seller_item_name'], valid_df['price']
candidates, candidate_prices = master_file['product_name_ar'], master_file['price']

In [3]:
p = Predictor()

scores = [p.candidate_ranking(
    query, candidates, query_prices[idx],
    candidate_prices, sort=True
) for idx, query in enumerate(queries)]         # shape: (queries num, candidates num, 3)

checkpoints/checkpoint_epoch_15.pth


In [4]:
valid_df = add_top_k_scores_to_df(valid_df, scores, master_file, k=3)

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Measures of the top 1 candidate is the correct one, doesn't account for 2nd and 3rd place.
print(f"Precision Score : {precision_score(valid_df['sku'], valid_df['sku1'], average='micro')}")
print(f"Recall Score    : {recall_score(valid_df['sku'], valid_df['sku1'], average='micro')}")
print(f"F1 Score        : {f1_score(valid_df['sku'], valid_df['sku1'], average='micro')}")

Precision Score : 0.985361653272101
Recall Score    : 0.985361653272101
F1 Score        : 0.985361653272101


In [6]:
# all the example where the correct one is not in the top 3
not_in_top3 = valid_df[(valid_df['sku'] != valid_df['sku1']) & (valid_df['sku'] != valid_df['sku2']) & (valid_df['sku'] != valid_df['sku3'])]
not_in_top3

,sku,marketplace_product_name_ar,seller_item_name,price,sku1,pred1,sim1,sku2,pred2,sim2,sku3,pred3,sim3
26,3683,اديمكس 1 مجم 20 قرص,اوميكاربكس 20 مجم اقراص,18.0,2355,اوفيوسيديك قطرة لزجة للعين 5 مل,0.576280,2470,وينديبين 5 مجم 30 قرص,0.457896,2359,ابيمول 300 مجم 5 اقماع,0.404508
156,1218,تيراتام اكس ار 500 مجم 20 قرص,فاست كير بى شامبو,148.0,12017,فيتاترون 30 كبسول,0.555839,1746,فورموهال 12 مكجم 30 كبسول للاستنشاق,0.384405,1031,تامسولين بلس 0 4 6 مجم 20 قرص ممتد المفعول,0.362273
390,2080,رومارين 75 مج 3 امبول,rheumarene amp 75 mg 3 3 ml,27.0,900,زيرتك 10 مجم مل نقط بالفم 10 مل,0.429193,2097,شاى ريجيم رويال 25 فلتر,0.333302,12653,محلول ملح وريد 0 9 500 مل انترافارم,0.333082
419,771,ميزوسيبت 1 امبول,mesocept 50 5 mg i m amp,23.5,1084,لاسيكس 2 مل 3 امبول,0.413702,1069,محلول ملح الفتح 0 9 500 مل,0.325369,2506,ريموواكس نقط للاذن 15 مل,0.317638
775,9390,بولي فريش اكسترا قطرة عين احادي الجرعة 30 0 4 مل,polyfresh extra drops 30 0 4 ml sdu,150.0,317,كوتارج 160 12 5 مجم 14 قرص,0.489258,71,برافوتين 100 مجم 14 كيس,0.454763,1189,بلوكاتنس 5 160 مجم 28 قرص,0.418235
908,941,اوركاديكسولين قطرة عين 5 مل,0 اوركادوكسيللين قطره,17.5,1017,نورمو تيرز قطرة عين 15 مل,0.480103,4305,رينكس 0 1 نقط انف للكبار 10 مل,0.463848,3501,يوريكونترول 2 مجم 10 قرص,0.368918
1252,1408,يوريبان 5 مجم 30 قرص,uripan tabs 5 mg 30,36.0,1778,جينوكونازول 0 8 كريم مهبلي 30 جم,0.244253,557,زيرتيك 0 1 شراب 100 مل,0.229958,40,زوراكسون 1 جم فيال عضل,0.220832
1342,1337,اكوا بلس شراب 100 مل,ميلوفلام 30 قرص,45.0,1304,رويال شاى ريجيم 50 فتلة,0.496943,1522,هيدروفيرين 50 مجم مل نقط بالفم 30 مل,0.476977,2402,سيلونا 20 كيس,0.451579
1462,523,سنجيولير 10 مجم 14 قرص,singulair tabs 10 mg 14,192.0,2413,مارنيز نيوكاربون 30 كبسول,0.287119,3640,سيكودال 3 مجم 30 قرص,0.213326,357,ديامونركتا 5 مجم 30 قرص,0.213149
1504,523,سنجيولير 10 مجم 14 قرص,singulair 10 mg 14 tab,192.0,2413,مارنيز نيوكاربون 30 كبسول,0.285058,3640,سيكودال 3 مجم 30 قرص,0.213703,357,ديامونركتا 5 مجم 30 قرص,0.209656


In [9]:
"""
Out of 7,000 validation examples, only 27 failed to include the correct candidate 
in the top three. This is primarily because some queries are in English while the 
candidate list is in Arabic.
"""

not_in_top3.shape[0]

27

In [16]:
save_dataframe_to_xlsx(
    df=valid_df,
    file_path='./validation_output.xlsx',
    sheet_name='Results'
)

DataFrame saved successfully to ./validation_output.xlsx
